In [70]:
import header
from header import __root__
from src import gs

In [71]:
import pandas as pd
import zipfile
import time
from pathlib import Path
from typing import Optional, List, Dict, Any
from types import SimpleNamespace
from src.utils.jjson import j_loads, j_loads_ns
from src.utils.file import read_text_file, recursively_yield_file_path
from src.utils.printer import pprint as print

In [72]:
class Config:
    """Класс конфигурации скрипта."""
    ENDPOINT: Path = __root__ / 'SANDBOX' / 'davidka'
    config: SimpleNamespace = j_loads_ns(ENDPOINT / 'davidka.json')
    STORAGE: Path
    try:
        STORAGE: Path = Path(config.local_storage.storage) if config.actual_storage == 'local_storage' else Path(config.google_drive.storage)
    except Exception as ex:
        raise ValueError(f"Ошибка загрузки пути к хранилищу: ", ex)
        
    TRAIN_STORAGE:Path = STORAGE / 'train_data' / 'product'
    UZZIPPED_STORAGE:Path = STORAGE / 'unzipped_storage' / 'product'
    GEMINI_API_KEY: Optional[str] = gs.credentials.gemini.onela.api_key # Установить имя владельца ключа (onela, kazazrinov, ...)
    GEMINI_MODEL_NAME: str = 'gemini-2.0-flash-exp' # Актуальное имя модели может отличаться, например, 'gemini-1.5-flash-latest'
    
    

In [73]:
"""
with zipfile.ZipFile("your_file.zip", 'r') as zip_ref:
    zip_ref.extractall(ENDPOINT / 'extracted_data')  # например, 'data'
    """

'\nwith zipfile.ZipFile("your_file.zip", \'r\') as zip_ref:\n    zip_ref.extractall(ENDPOINT / \'extracted_data\')  # например, \'data\'\n    '

In [74]:
print(Config.TRAIN_STORAGE)

E:\hypo69\llm\train_data\product


In [101]:
"""
products_list_all: list[dict] = []
counter: int = 0

for file in recursively_yield_file_path(Config.TRAIN_STORAGE, '*.json'):
    try:
        products_list_from_file: list[dict] = j_loads(file)
        products_list_all.extend(products_list_from_file)
        counter += 1
        #print(f"{counter}: {file} — {len(products_list_from_file)} items")
    except Exception as ex:
        logger.error(f"Ошибка при чтении {file}", ex)
    """

SyntaxError: incomplete input (1402321004.py, line 1)

In [91]:
#print(len(products_list_all))


1965


In [92]:
"""
df = pd.DataFrame()

for product in products_list_all:
    df = pd.concat([df, pd.DataFrame([product])], ignore_index=True)
"""

'\ndf = pd.DataFrame()\n\nfor product in products_list_all:\n    df = pd.concat([df, pd.DataFrame([product])], ignore_index=True)\n'

In [102]:
products_dict: dict = {}
counter: int = 0

for file in recursively_yield_file_path(Config.TRAIN_STORAGE, '*.json'):
    try:
        products_dict.update( j_loads(file) )
        counter += 1
        #print(f"{counter}: {file} — {len(products_list_from_file)} items")
    except Exception as ex:
        logger.error(f"Ошибка при чтении {file}", ex)

In [103]:
df = pd.DataFrame(products_dict)

In [104]:
print(len(df))

20


In [ ]:
df.head

In [96]:
df.columns[:2]

RangeIndex(start=0, stop=1, step=1)

In [97]:
print(len(df.columns))

1


In [98]:
df_transposed = df.T

In [99]:
df_transposed.columns

RangeIndex(start=0, stop=1965, step=1)

In [100]:
df_transposed.head

<bound method NDFrame.head of                                                 0     \
0  https://airgasnordest.com/wp-content/uploads/T...   

                                                1     \
0  https://airgasnordest.com/abrasivi/lpg-ceramic...   

                                                2     \
0  https://airgasnordest.com/impianti_da_taglio/m...   

                                                3     \
0  https://airgasnordest.com/impianti_da_taglio/m...   

                                  4     \
0  https://airgasnordest.com/noleggio/   

                                                5     \
0  https://airgasnordest.com/abrasivi/norton-razo...   

                                                6     \
0  https://airgasnordest.com/wp-content/uploads/V...   

                                                7     \
0  https://airgasnordest.com/wp-content/uploads/R...   

                                                8     \
0  https://airgasnordest.com/impianti_

In [ ]:
df_cleaned = df_transposed.dropna(subset=['supplier'])


In [ ]:
supplier_list = df_cleaned['supplier'].dropna().unique()
print(supplier_list)

In [ ]:
supplier_counts = df_clean.groupby('supplier').count().reset_index()

In [ ]:
print(supplier_counts)

In [ ]:
df_transposed.columns

## В следующей функции возникает ошибка скорее всего из-за неправильного символа UNICODE в названии столбца

In [ ]:
columns_to_check = ['category']
actual_category_column_name = None
for c_name in df_transposed.columns:
    if 'category' in c_name: # или c_name == 'category'
        actual_category_column_name = c_name
        break

if actual_category_column_name:
    columns_to_check = [actual_category_column_name]
else:
    columns_to_check = [] # 'category' не найдена

existing_columns = [col for col in columns_to_check if col in df_transposed.columns]


# Если колонки существуют, считаем товары
if existing_columns:
    # Объединяем значения из существующих колонок
    category_counts = pd.concat([df[col] for col in existing_columns])

    # Считаем количество товаров в каждой категории
    category_counts = category_counts.dropna().value_counts().reset_index()
    category_counts.columns = ['category', 'product_count']
else:
    category_counts = pd.DataFrame(columns=['category', 'product_count'])

print(category_counts)

In [ ]:
df_transposed.to_csv(Config.TRAIN_STORAGE / 'tarin.csv', index=False, encoding='utf-8')